## Playing around with Spar Nord open data
Spar Nord has currently opened up access to:
* ATM withdrawals in 2017

In [1]:
import pandas as pd
import glob
from requests import get  # to make GET request

# Download and unzip file
def download(url, file_name, directory):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = get(url)
        # write to file
        file.write(response.content)
    import zipfile
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(directory)

download("https://sparnordopenbanking.com/downloads/open_dataset.zip", "file.zip","open_dataset")

# Importing multiple csv datafiles to Pandas dataframe
path =r'./open_dataset/' # use your path
allFiles = glob.glob(path + "/*.csv")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
# Append all files to Pandas dataframe
data = pd.concat(list_, axis = 0, ignore_index = True)

In [2]:
data[:10]

,year,month,day,weekday,hour,atm_status,atm_id,atm_manufacturer,atm_location,atm_streetname,...,temp,pressure,humidity,wind_speed,wind_deg,rain_3h,clouds_all,weather_id,weather_main,weather_description
0,2017,July,2,Sunday,12,Active,41,Diebold Nixdorf,Skagen,Sct. Laurentiivej,...,287.974082,1009,77,8,257,NaN,81,803,Clouds,broken clouds
1,2017,July,2,Sunday,12,Active,8,NCR,Glyngøre,Færgevej,...,287.776783,1016,85,9,281,NaN,52,803,Clouds,broken clouds
2,2017,July,2,Sunday,12,Inactive,12,NCR,Østerå Duus,Østerå,...,288.756852,1020,71,11,276,NaN,25,801,Clouds,few clouds
3,2017,July,2,Sunday,12,Inactive,52,NCR,Farsø,Torvet,...,288.756852,1020,71,11,276,NaN,25,801,Clouds,few clouds
4,2017,July,2,Sunday,12,Active,48,Diebold Nixdorf,Brønderslev,Algade,...,288.756852,1020,71,11,276,NaN,25,801,Clouds,few clouds
5,2017,July,2,Sunday,12,Active,33,NCR,Vadum,Ellehammersvej,...,288.756852,1020,71,11,276,NaN,25,801,Clouds,few clouds
6,2017,July,2,Sunday,12,Active,84,NCR,Svendborg,Sankt Nicolai Gade,...,289.390441,1017,88,6,285,NaN,35,802,Clouds,scattered clouds
7,2017,July,2,Sunday,12,Inactive,88,NCR,Storcenter indg. A,Hobrovej,...,288.756852,1020,71,11,276,NaN,25,801,Clouds,few clouds
8,2017,July,2,Sunday,12,Active,65,NCR,Storvorde,Vandværksvej,...,288.756852,1020,71,11,276,NaN,25,801,Clouds,few clouds
9,2017,July,2,Sunday,12,Active,31,NCR,Slagelse,Mariendals Alle,...,289.237276,1016,79,7,271,NaN,39,802,Clouds,scattered clouds


### Lets start by plotting ATM´s on a map to see actual locations
* Lat and longs need to be converted to Web Mercator measures<br>
* Plotting using Bokeh library

In [2]:
import math
from pandasql import sqldf
from tqdm import tqdm
top = 25000
# Lat / Longs need to be converted to mercator measures before plotting
def geographic_to_web_mercator(x_lon, y_lat):
    if abs(x_lon) <= 180 and abs(y_lat) < 90:
        num = x_lon * 0.017453292519943295
        x = 6378137.0 * num
        a = y_lat * 0.017453292519943295
        x_mercator = x
        y_mercator = 3189068.5 * math.log((1.0 + math.sin(a)) / (1.0 - math.sin(a)))
        return x_mercator, y_mercator
    else:
        print('Invalid coordinate values for conversion')        

        
# Group ATM's, counting number of withdrawals
grouped = sqldf('select atm_lat, atm_lon, count(*) as numberOfWithdrawals from data group by atm_lat, atm_lon')

# Create lists and convert lan/longs
lats=grouped['atm_lat'][:top].tolist()
longs=grouped['atm_lon'][:top].tolist()
withdrawals=grouped['numberOfWithdrawals'][:top].tolist()

mercator_lats = []
mercator_longs = []
for i, lat in tqdm(enumerate(lats)):
    mercator_long, mercator_lat = geographic_to_web_mercator(longs[i],lat)
    mercator_lats.append(mercator_lat)
    mercator_longs.append(mercator_long)

92it [00:00, 101226.64it/s]


In [3]:
#plot the map
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import output_notebook
# Allign the map to show only Denmark
left   =  800000
right  = 1800000
bottom = 7350000
top    = 8000000

output_file("tile.html")
output_notebook()
# range bounds supplied in web mercator coordinates
p = figure(x_range=(left, right), y_range=(bottom, top))
# Add the map tile
p.add_tile(CARTODBPOSITRON)
# Plot the ATMs
p.circle(x=mercator_longs, y=mercator_lats, alpha = 2, size = 5)
show(p)

Loading BokehJS ...